# AI504 Project 2

## To-Do : Find better hyperparameters
The goal of this project is improving the performance of Neural Machine Translation(NMT) system. In this project, you will tune the hyperparameters to achieve higher BLEU score without changing anything else (e.g. architecture, dataset, etc)

In [5]:
from easydict import EasyDict

config = EasyDict({
    "emb_dim":64,
    "ffn_dim":128,
    "attention_heads":8,
    "dropout":0.2568,
    "encoder_layers":3,
    "decoder_layers":2,
    "lr":2.4963e-3,
    "batch_size":4610,
    "nepochs":48,
})

#####      Do not modify      #####
config.max_position_embeddings=512

## Download files
Before execute this code, you should run the template codes first. This code will automatically downloads the state_dict of your model and configuration file which you use for training & evaluation.

Please change the student ID before you run this.

__CAUTION__ : Please run this code with *Google Chrome* browser. 

In [8]:
from google.colab import files
import os

os.environ['STUDENT_ID']="20204871"

if os.path.isdir('result'):
  !rm -rf result

%mkdir result
%mv config.json model.pt result

!zip $STUDENT_ID.zip result/*
# files.download('{}.zip'.format(os.environ['STUDENT_ID']))

  adding: result/config.json (deflated 31%)
  adding: result/model.pt (deflated 8%)


## Template codes (do not modify)
This code is equivalent to the code in [Week 11](https://classum.com/main/course/7726/111). Please refer to codes & descriptions in a link for details.

### Data loader

In [2]:
!pip install --upgrade torchtext
!python -m spacy download de
!python -m spacy download en
!pip install -Iv --upgrade nltk==3.5

import torch
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

torch.manual_seed(1234)
torch.cuda.manual_seed_all(1234)

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

SRC.build_vocab(train_data, min_freq = 3)
TRG.build_vocab(train_data, min_freq = 3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = config.batch_size,
    device = device,
    shuffle=False)

PAD_IDX = TRG.vocab.stoi['<pad>']

     |████████████████████████████████| 6.9MB 4.6MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
     |████████████████████████████████| 14.9MB 716kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=fd75c92171ee78ed4c7b8bbea3923bf86aa3126032391b44f4182cf74712c91c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fo34tgha/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/di

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 323kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 91.8kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 86.3kB/s]
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


### Load model & optimizer

In [6]:
import torch.nn as nn
import torch.optim as optim

class Transformer(nn.Module):
    def __init__(self, config):
        super(Transformer,self).__init__()
        self.encoder_embedding = nn.Embedding(len(SRC.vocab),config.emb_dim)
        self.decoder_embedding = nn.Embedding(len(TRG.vocab),config.emb_dim)
        self.transformer = nn.Transformer(d_model=config.emb_dim, nhead=config.attention_heads, 
                       num_encoder_layers=config.encoder_layers, num_decoder_layers=config.decoder_layers,
                       dim_feedforward=config.ffn_dim, dropout=config.dropout, activation='gelu')
        self.prediction_head = nn.Linear(config.emb_dim,len(TRG.vocab))
        
    def forward(self, src, trg):
        src_emb = self.encoder_embedding(src)
        trg_emb = self.decoder_embedding(trg)
        output = self.transformer(src_emb, trg_emb,
                       tgt_mask=self.transformer.generate_square_subsequent_mask(trg.size(0)).to(device),
                       src_key_padding_mask=src.eq(PAD_IDX).permute(1,0).to(device),
                       memory_key_padding_mask=src.eq(PAD_IDX).permute(1,0).to(device),
                       tgt_key_padding_mask=trg.eq(PAD_IDX).permute(1,0).to(device))
        prediction = self.prediction_head(output)
        return prediction

CLIP = 1
    
model = Transformer(config)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

### Train & Evaluation

In [7]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from tqdm import tqdm
import json

best_valid_loss = float('inf')

def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):
    model.train()

    epoch_loss = 0

    for idx, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[:-1].reshape(-1, output.shape[-1])
        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):
    model.eval()

    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg)            
            
            output = output[:-1].reshape(-1, output.shape[-1])
            
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def measure_BLEU(model: nn.Module,
             iterator: BucketIterator
                ):
    model.eval()
    iterator.batch_size = 1
    BLEU_scores = list()
    
    with torch.no_grad():
        for idx, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg)           
            predicted = [TRG.vocab.itos[token] for token in output[:-1].argmax(dim=2).squeeze().tolist() if token!=PAD_IDX]
            GT = [TRG.vocab.itos[token] for token in trg[1:].squeeze().tolist() if token!=PAD_IDX]
            BLEU_scores.append(sentence_bleu([GT], predicted))
    return sum(BLEU_scores)/len(BLEU_scores)
                         
queue=0
for epoch in tqdm(range(config.nepochs), total=config.nepochs):
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    test_bleu = measure_BLEU(model, test_iterator)
    print("Test BLEU score : {}".format(test_bleu))
    print("Epoch : {} / Training loss : {} / Validation loss : {}".format(epoch+1, train_loss, valid_loss))

    if best_valid_loss < valid_loss:
        queue+=1
        if queue>1:
            break
    else:
        best_valid_loss = valid_loss
        queue = 0

test_bleu = measure_BLEU(model, test_iterator)
print("Test BLEU score : {}".format(test_bleu))
        
with open('config.json','w') as f:
    json.dump(vars(config),f)
torch.save(model.state_dict(),'model.pt')

  0%|          | 0/48 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of l

Test BLEU score : 0.010262044585956578
Epoch : 1 / Training loss : 5.362898629809183 / Validation loss : 4.265905380249023


  4%|▍         | 2/48 [00:47<18:06, 23.61s/it]

Test BLEU score : 0.027407518816389028
Epoch : 2 / Training loss : 4.0404180118015836 / Validation loss : 3.7211442788441977


  6%|▋         | 3/48 [01:10<17:42, 23.61s/it]

...

 67%|██████▋   | 32/48 [12:29<06:15, 23.45s/it]

Test BLEU score : 0.23946872343162895
Epoch : 32 / Training loss : 1.7305705452722215 / Validation loss : 2.271376132965088


 67%|██████▋   | 32/48 [12:52<06:26, 24.15s/it]

Test BLEU score : 0.24151288171215976
Epoch : 33 / Training loss : 1.7156043771713498 / Validation loss : 2.268957257270813


Test BLEU score : 0.24151288171215976
